# An advanced look at LLM Agents

Previously, we relied on LangChain to to build the agent. However, this is not necessary, since an agent is nothing more than a fancy while loop. 

**The goal** 

With this notebook you will see what an agent is under the hood, and you can build it based on the LLM output

🌟 So ... let us begin!  

# Install dependencies

Compile the cell below to install the dependencies. Consider clearing the cell output so it does not clutter your notebook.

In [ ]:
!python3 -m venv venv         
!source venv/bin/activate     
!pip3 install -r helper_functions/requirements.txt

# The key elements

The key elements of the agent remain - we need tools, an LLM that functions as the agent and a prompt with the query. In our example, we'll use ``chatgpt35-turbo`` as the agent LLM, [here](https://platform.openai.com/docs/guides/text-generation/chat-completions-api) you find the documentation on how to query the model through the OpenAI API.

# The tools

**Exercise:** Compile the cell below and check out the description. How does it differ from the previous tool description?

In [ ]:
from helper_functions.helper_functions import simple_description_formatter
from helper_functions.tools import my_own_wiki_tool, weather_tool
import pprint

# load the tools and format. In plain OpenAI jargon they are called functions.
tools = [my_own_wiki_tool, weather_tool]
function_description = [simple_description_formatter(tool) for tool in tools]
pprint.pprint(function_description[0])

# Store executable functions with their name in dictionary
available_functions = {tool.name: tool for tool in tools}


# The LLM output

the prompt is feed as part of the messages object. All input, such as the system prompt, the user question, possibly also chat-history is provided through this object.

**Exercise:** compile both questions, and compare the answers. Do you see a difference?

Hint: The LLM can output two types of answers:
* a string that answers the question, 
* a function-call object, which contains information on which function to call with which arguments.

In [ ]:

from helper_functions.keys import client


system_prompt = "You are a friendly, helpful assistant. Your goal is to answer the questions in a concise, but conversational manner."

questions = ["what is the meaning of life?","How many people live in Paris?"]

for question in questions:
  messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": question}
    ]

  
  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    tools = function_description,
    messages=messages, 
    )

  print(f"Question: {question}")
  print(f"Answer: {response.choices[0].message.content}")
  print(f"Fuction call: {response.choices[0].message.tool_calls}\n")



# The Agent - a fancy while loop

While the LLM requests function calls we 
* per function, **extract** the **name and arguments** to be called from the initial LLM response,
* **execute** the **function calls**,
* **store** the **output of the function** in the messages object,
* invoke the LLM again, until no function call are requested.

For more details, you can also check out this [OpenAI function calling guide](https://platform.openai.com/docs/guides/function-calling).

**Exercise:** Change the question and investigate the output. Do you understand what you see:

In [27]:
import json

# question = "How many people live in Paris?"
# question = "which city is bigger: Paris or Munich?"
question = 'Tell me some juicy details of the king of the netherlands'

messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": question}
    ]

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  tools = function_description,
  messages=messages, 
  )

print('==== Initial LLM response ====')
print(f"Answer: {response.choices[0].message.content}")
print(f"Function call: {response.choices[0].message.tool_calls}\n")

# while the response requests function calls
while response.choices[0].message.tool_calls:
    
  # store response message with all function calls
  response_message = response.choices[0].message
  messages.append(response_message)

  # execute each tool individually
  for tool_call in response.choices[0].message.tool_calls:
    print('==== Function call ====')

    # function name and arguments
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)
    print(f'Calling function "{function_name}" with arguments {function_args}.')

    # execute function call 
    function_response = available_functions[function_name].invoke(function_args)
    print(f'Function call response:\n{function_response}\n')

    # append function response to messages
    messages.append({
        "tool_call_id":tool_call.id, 
        "role": "tool", 
        "name": function_name, 
        "content": function_response
    })
    
  # get a new response from LLM
  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    tools = function_description,
    messages=messages, 
  )

  print('== Intermediate LLM response ==')
  print(f"Answer: {response.choices[0].message.content}")
  print(f"Function call: {response.choices[0].message.tool_calls}\n")

print('==== Final LLM response ====')
print("Question: ", question)
print(f"Answer: {response.choices[0].message.content}")
print(f"Function call: {response.choices[0].message.tool_calls}\n")

==== Initial LLM response ====
Answer: None
Function call: [ChatCompletionMessageToolCall(id='call_QiDKtvRh1w29vxbwATS2TECo', function=Function(arguments='{"query":"King of the Netherlands"}', name='wikipedia'), type='function')]

==== Function call ====
Calling function "wikipedia" with arguments {'query': 'King of the Netherlands'}.
Function call response:
Page: Monarchy of the Netherlands
Summary: The monarchy of the Netherlands is a constitutional monarchy whose role and position are governed by the Constitution of the Netherlands. Roughly a third of the Constitution explains the succession, mechanisms of accession and abdication to the throne, the roles and duties of the monarch, the formalities of communication between the States General of the Netherlands, and the monarch's role in creating laws.
The Kingdom of the Netherlands has been an independent monarchy since 16 March 1815. Its once-sovereign provinces had been intermittently ruled by members of the House of Orange-Nassau 